In [11]:
import numpy as np
import tensorflow as tf
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use('TkAgg')
np.set_printoptions(suppress=True)

#### Load dataset

In [12]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()
print("Train = ", x_train.shape, y_train.shape)
print("Test = ", x_train.shape, y_test.shape)
plt.imshow(x_train[10],cmap='gray')
plt.show()

Train =  (60000, 28, 28) (60000,)
Test =  (60000, 28, 28) (10000,)


#### Missing values

In [13]:
print("Train (Missing values) = ", np.isnan(x_train).any(), ",", np.isnan(y_train).any())

Train (Missing values) =  False , False


#### Categorical/Text features

In [14]:
print("Train (Datatype) = ", x_train.dtype, ",", x_test.dtype)

Train (Datatype) =  uint8 , uint8


#### Feature scaling (Normalization)

In [15]:
x_train_temp = np.reshape(x_train, (x_train.shape[0], -1))
x_test_temp = np.reshape(x_test, (x_test.shape[0], -1))
print(pd.DataFrame(x_train_temp).describe())

                0             1             2             3             4    \
count  60000.000000  60000.000000  60000.000000  60000.000000  60000.000000   
mean       0.000800      0.005783      0.030083      0.103800      0.249683   
std        0.092554      0.249033      0.767868      2.512017      4.331376   
min        0.000000      0.000000      0.000000      0.000000      0.000000   
25%        0.000000      0.000000      0.000000      0.000000      0.000000   
50%        0.000000      0.000000      0.000000      0.000000      0.000000   
75%        0.000000      0.000000      0.000000      0.000000      0.000000   
max       16.000000     36.000000    119.000000    164.000000    224.000000   

                5             6             7             8             9    \
count  60000.000000  60000.000000  60000.000000  60000.000000  60000.000000   
mean       0.414717      0.821667      2.224733      5.698667     14.434650   
std        5.827394      8.309935     14.201820    

In [16]:
x_train = x_train/255 #(x_train-0)/(255-0)
x_test = x_test/255 #(x_test-0)/(255-0)
print("Normalized => Min (Train) = ",np.min(x_train)," and Max (Train) = ",np.max(x_train))

Normalized => Min (Train) =  0.0  and Max (Train) =  1.0


#### Build MODEL

In [17]:
callback_es = tf.keras.callbacks.EarlyStopping(monitor='loss',patience=3)
class AccCheck(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        if logs['accuracy'] >= 0.95:
            print("Reached 99.5% accuracy so cancelling training!")
            self.model.stop_training = True

fm_model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(filters=64,kernel_size=3,activation='relu',input_shape=(28,28,1)),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Conv2D(filters=32,kernel_size=3,activation='relu'),
    tf.keras.layers.MaxPool2D(),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=128, activation='relu'),
    tf.keras.layers.Dense(units=10, activation='softmax')
])

fm_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [18]:
fm_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 26, 26, 64)        640       
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 13, 13, 64)       0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 11, 11, 32)        18464     
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 5, 5, 32)         0         
 2D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 800)               0         
                                                                 
 dense_2 (Dense)             (None, 128)              

In [19]:
fm_model.fit(x_train, y_train, epochs=10, callbacks=[callback_es,AccCheck()])

Epoch 1/10
1875/1875 [==============================] - 39s 21ms/step - loss: 0.4554 - accuracy: 0.8351
Epoch 2/10
1875/1875 [==============================] - 35s 19ms/step - loss: 0.3077 - accuracy: 0.8887
Epoch 3/10
1875/1875 [==============================] - 35s 19ms/step - loss: 0.2631 - accuracy: 0.9028
Epoch 4/10
1875/1875 [==============================] - 35s 19ms/step - loss: 0.2368 - accuracy: 0.9113
Epoch 5/10
1875/1875 [==============================] - 35s 19ms/step - loss: 0.2108 - accuracy: 0.9208
Epoch 6/10
1875/1875 [==============================] - 34s 18ms/step - loss: 0.1923 - accuracy: 0.9278
Epoch 7/10
1875/1875 [==============================] - 36s 19ms/step - loss: 0.1748 - accuracy: 0.9344
Epoch 8/10
1875/1875 [==============================] - 578s 308ms/step - loss: 0.1583 - accuracy: 0.9402
Epoch 9/10
1875/1875 [==============================] - 38s 21ms/step - loss: 0.1453 - accuracy: 0.9454
Epoch 10/10
1875/1875 [==============================] - 36s 1

#### Evaluate

In [22]:
print("Loss, Accuracy (Test) = ", fm_model.evaluate(x_test, y_test))

313/313 [==============================] - 2s 7ms/step - loss: 0.2715 - accuracy: 0.9126
Loss, Accuracy (Test) =  [0.2714907228946686, 0.9125999808311462]


#### Predict

In [23]:
def predict_labels(pred_arr):
    idx = np.argmax(pred_arr, axis=1)  # Row-wise
    labels = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']
    return [labels[i] for i in idx]

y_pred = fm_model.predict(x_test)
print("Predictions (sample) = ", predict_labels(y_pred[15:20]))

313/313 [==============================] - 2s 6ms/step
Predictions (sample) =  ['Trouser', 'Pullover', 'Coat', 'Bag', 'T-shirt/top']
